In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 13619}


13619

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 21873}


21873

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-28T13:07:15.528000Z,a2d57e34-e4fc-4bf2-8bc5-60be3eeedb7f,1,Leonce,Jocelaine
1,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-28T13:05:26.748000Z,2d5056ef-93dc-4acb-a602-dfa8a3e6f9c6,3,Leonce,Linda
2,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-28T13:03:23.027000Z,53a4a6d3-0500-471e-909a-6bb4956b7d8a,2,Silencieux,Sonel
3,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-28T13:01:32.091000Z,20d53417-2bd0-46bf-b193-4377799945b6,3,Fany,Gertride
4,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-28T12:59:36.323000Z,7f46f969-6c2b-4255-8e69-24dd5a54e559,2,Alsaint,Maquise
...,...,...,...,...,...,...,...
12780,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
12781,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
12782,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
12783,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


12785

In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,02a25cddec634621822ccaa4657f8246,2d6e3a4b-5a01-4f4f-9a69-b075aadac39f,3,1,,,b58699f1-47c2-4b05-ac0d-9a3034328010,child
1,8fd7f28e6c31484993c3153565feb429,ba32857a-7735-47b5-b9a6-abeb57b99ea9,3,2,non,non,cd1e58fb9bca47509b58e1e0c51e47e3,child
2,ece16e9f4d2c468e961d2b56ff621a2e,a253548c-4fbb-4ab2-b316-e2584e21c212,17,2,,,8b1013b7-287c-48ef-b123-f5bc451b7de9,child
3,8fd7f28e6c31484993c3153565feb429,36be1e5d-f2bf-4ec3-b35f-5e56e88aa8b6,17,1,non,non,6eb7b97bf219409c843d64f23f677c6b,child
4,ece16e9f4d2c468e961d2b56ff621a2e,0d34b807-6d7e-4190-9680-53bbf372df1a,11,2,,,995a88f0-2578-4c9d-a91c-57399e82557a,child
...,...,...,...,...,...,...,...,...
21868,bfe6fcc7ba4f4c73899c3641fe40d905,f82fab40-61d8-4f53-b746-9a105a038ca0,8,2,non,non,53a4a6d3-0500-471e-909a-6bb4956b7d8a,child
21869,bfe6fcc7ba4f4c73899c3641fe40d905,c2411746-e04d-4e5c-b1f1-0a727cc802fa,8,2,oui,oui,2d5056ef-93dc-4acb-a602-dfa8a3e6f9c6,child
21870,bfe6fcc7ba4f4c73899c3641fe40d905,c01454e3-6c98-4ca5-87e0-633c2a646afb,5,1,oui,oui,2d5056ef-93dc-4acb-a602-dfa8a3e6f9c6,child
21871,bfe6fcc7ba4f4c73899c3641fe40d905,f9bcb417-497b-440c-8be6-f23875c1ab36,10,1,oui,oui,2d5056ef-93dc-4acb-a602-dfa8a3e6f9c6,child


21873

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [8]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.qty.sum()
result_hivmuso.reset_index(inplace=True)
result_hivmuso


,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",399
1,"(Gustave, firma.gustave@carisfoundationintl.org)",1049
2,"(June007, jeanrony.valbrun@carisfoundationintl...",979
3,"(despeignes, roselord.despeignes@carisfoundati...",367
4,"(jfodney17, fodney.joseph@carisfoundationintl....",895
5,"(josy22, josy.serafils@carisfoundationintl.org)",510
6,"(judesonvOLMAR, judeson.volmar@carisfoundation...",384
7,"(maelisma, mcarthur.elisma@carisfoundationintl...",17
8,"(marcakernstdubois, kernst.dubois@carisfoundat...",722
9,"(robentz, pierrerobentz.cassion@carisfoundatio...",8783
